<h2><b>Note: </b>I will address all Three parts of the assignment in this notebook</h2>

<h1><b>1st Part: Extracting the data</b></h1>
<h5>In this part I will address the first part of the assignments 
where I will use "BeautifulSoup" library to extract the niegborhoods table</br>
from the Wikipedia page and put them int a pandas DataFrame execluding 
the Not Assigned Bouroughs as mentioned in the assignment</h5>

In [7]:
#!pip install bs4
#!pip install folium
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import folium

In [8]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html')
neighborhood_tabal = soup.find('table',{'class':'wikitable sortable'})
ColumnsNames = []
columns_names_holder = neighborhood_tabal.findAll('th')
for name in columns_names_holder:
    ColumnsNames.append(name.text.rstrip())

In [9]:
toronto_neighborhood = pd.DataFrame(columns = ColumnsNames)

In [10]:
values = neighborhood_tabal.findAll('td')
length = len(values)
i = 0
while i < length:
    neigborhood = values[i + 2].text.rstrip()
    if neigborhood == 'Not assigned':
        neigborhood = values[i + 1].text.rstrip()        
    toronto_neighborhood = toronto_neighborhood.append({'Postcode': values[i].text.rstrip(), 'Borough' : values[i+1].text.rstrip() , 'Neighborhood' : values[i+2].text.rstrip()},ignore_index = True)
    i = i + 3
Toronto = toronto_neighborhood[toronto_neighborhood['Borough'] != 'Not assigned']
Toronto.reset_index(drop=True,inplace = True)
#https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value
Toronto =  Toronto.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
Toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
Toronto.shape[0]

103

<h1><b>2nd Part: Getting the Latitude and Longitude</b></h1>
<h5>In this part I will use the file provided in assignments to get the latitude and longitude for each
Neigborhood</h5>

In [12]:
#!wget 'https://cocl.us/Geospatial_data'
df = pd.read_csv('Geospatial_data')
df.set_index('Postal Code',inplace = True)
Toronto['Latitude'] =''
Toronto['Longitude'] = ''
for i in range(Toronto.shape[0]):
    Toronto.at[i,'Latitude'] = df.loc[Toronto.loc[i,'Postcode'],'Latitude']
    Toronto.at[i,'Longitude'] = df.loc[Toronto.loc[i,'Postcode'],'Longitude']

<h4>Here I filter the dataframe to contain only boroughs that contains the word "Toronto"</h4>

In [13]:
myFilter = Toronto['Borough'].str.contains(pat = 'Toronto')
Toronto = Toronto[myFilter]
Toronto.reset_index(inplace = True)
Toronto.drop('index',axis = 1,inplace = True)
Toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6764,-79.293
1,M4K,East Toronto,"The Danforth West,Riverdale",43.6796,-79.3522
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.669,-79.3156
3,M4M,East Toronto,Studio District,43.6595,-79.3409
4,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


<h1><b>3rd Part: CLustering and visualization</b></h1>
<h5>In this part I replicate the work done in New York Neighborhood Lab with some minor changes</h5>

In [16]:
CLIENT_ID = 'GYDUYW4SB4BUA2R3I0AZ5CKUVOWLPSHETFFY31NEHKTOT0I5'
CLIENT_SECRET = 'OCKBDLCLFXAKZP0XPQBKIKUKQFGO1HS1M0RVTIS333KP5CLB'
VERSION = '20180605'
LIMIT = 100
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4>In here I save the Data Frame into a csv file after getting in order to avoid wasting 
many calls to the Foursquare API if I have to repeat the process multiple time</h4>

In [17]:
Toronto_Venues = getNearbyVenues(names=Toronto['Postcode'],latitudes = Toronto['Latitude'],longitudes=Toronto['Longitude'])
print(Toronto_Venues.shape[0])
Toronto_Venues.head()
Toronto_Venues.to_csv('Toronto_Venues',index = False)
#Toronto_Venues = pd.read_csv('Toronto_Venues')
#Toronto_Venues.head()

1670


In [18]:
toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(0,'Postcode',Toronto_Venues['Postcode'])
toronto_onehot.head()

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.00,...,0.00000,0.000000,0.023810,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048780,0.000000,0.00,...,0.02439,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.024390,0.000000,0.024390
4,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
7,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
import numpy as np
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M4E,Other Great Outdoors,Pizza Place,Health Food Store,Coffee Shop,Trail,Neighborhood,Pub
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Bookstore,Restaurant
2,M4L,Sandwich Place,Park,Ice Cream Shop,Food & Drink Shop,Brewery,Burger Joint,Burrito Place
3,M4M,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Convenience Store
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space


In [36]:
from sklearn.cluster import KMeans
kclusters = 4
toronto_grouped_feature_set = toronto_grouped.drop('Postcode', 1)
#kmeans = KMeans(init = "k-means++",n_clusters=kclusters, random_state=0).fit(toronto_grouped_feature_set)
toronto_grouped

,Postcode,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.00,...,0.00000,0.000000,0.023810,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048780,0.000000,0.00,...,0.02439,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.024390,0.000000,0.024390
4,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
7,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.00,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000


In [37]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#print(len(kmeans.labels_))
toronto_merged = Toronto

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')
toronto_merged = toronto_merged.astype({'Cluster Labels' : int})


0     0.0
1     0.0
2     0.0
3     0.0
4     2.0
5     2.0
6     0.0
7     0.0
8     3.0
9     0.0
10    3.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    1.0
23    2.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    NaN
Name: Cluster Labels, dtype: float64

In [27]:
import matplotlib.cm as cm
import matplotlib.colors as colors

toronto_latitude = 43.6532
toronto_longitude = -79.3832
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float